In [1]:
import pandas as pd
from typing import List, Tuple
import numpy as np
from anytree import node
from collections import deque, namedtuple
from itertools import product

In [2]:
def establish_board(board_size:int) -> List[List]:
    return np.array([['*']*board_size]*board_size)

In [3]:
board = establish_board(4)
board[0][2] = 'Q'
board[1][0] = 'Q'
board[2][0] = 'R'
board[2][3] = 'Q'
#board[0][3] = 'Q'
board

array([['*', '*', 'Q', '*'],
       ['Q', '*', '*', '*'],
       ['R', '*', '*', 'Q'],
       ['*', '*', '*', '*']], dtype='<U1')

In [4]:
def check_collision(collision_chars: List[str], check_spaces_lists:List[List[str]]) -> Tuple[bool, str]:
    collisions = []
    for check_spaces_list in check_spaces_lists:
        for space in check_spaces_list:
            if space in collision_chars:
                collisions.append(space)
                break
    return len(collisions) != 0, collisions

def get_collision_spaces(board: np.array, row: int, col: int) -> List[List[str]]:
    return_lists = []
    board_width = board.shape[0]
    board_height = board.shape[1]
    # DOWN
    return_lists.append(
        [board[i][col] for i in range(row + 1, board_height)]
    )
    # UP
    return_lists.append(
        [board[i][col] for i in reversed(range(0, row))]
    )
    # RIGHT
    return_lists.append(
        [board[row][j] for j in range(col + 1, board_width)]
    )
    # LEFT
    return_lists.append(
        [board[row][j] for j in reversed(range(0, col))]
    )

    # NOTE: all ranges need to START from the point and grow outwards, thus the use of 
    #       reversed in several cases

    # DOWN_RIGHT
    return_lists.append(
        [board[i][j] for i, j in zip(range(row + 1, board_height), range(col + 1, board_width))]
    )
    # DOWN_LEFT
    return_lists.append(
        [board[i][j] for i, j in zip(range(row + 1, board_height), reversed(range(0, col)))]
    )
    # UP_LEFT
    return_lists.append(
        [board[i][j] for i, j in zip(reversed(range(0, row)), reversed(range(0, col)))]
    )   
    # UP_RIGHT
    return_lists.append(
        [board[i][j] for i, j in zip(reversed(range(0, row)), range(col + 1, board_width))]
    ) 

    return return_lists

def is_valid_placement(board: np.array, potential_row: int, potential_col: int, blockers: List[str], invalid_blockers: List[str]):
    
    is_blocked, active_blockers = check_collision(
        blockers,
        get_collision_spaces(board, potential_row, potential_col)
    )

    problem_blockers = list(set.intersection(set(active_blockers), set(invalid_blockers)))

    return len(problem_blockers) == 0, active_blockers

def is_space_occupied(board: np.array, row: int, col: int) -> bool:
    return board[row][col] != '*'

In [5]:
board[0][0] == ' '

False

In [6]:
print(get_collision_spaces(board, 3, 0))
print(check_collision(['Q', 'R'], get_collision_spaces(board, 3, 0)))


[[], ['R', 'Q', '*'], ['*', '*', '*'], [], [], [], [], ['*', '*', '*']]
(True, ['R'])


In [7]:
is_valid_placement(
    board,
    3,
    0,
    ['Q', 'R'],
    ['Q']
)

(True, ['R'])

In [8]:
board_state = namedtuple("board_state", "board,num_placed_queens")

def place_queens(
        starting_board: np.array, 
        n_queens: int, 
        blocking_chars: List[str], 
        invalid_blockers: List[str],
        piece_value: str = 'Q',
    ) -> List[np.array]:

    board_height = starting_board.shape[0]
    board_width = starting_board.shape[1]

    potential_solutions = deque()
    potential_solutions.append(board_state(starting_board.copy(), 0))

    encountered_boards = {} # likely more efficient way to do this

    solutions=[]

    while len(potential_solutions) > 0:
        
        # pop from deque

        solution_step = potential_solutions.pop()

        # check if we have a valid solution

        if solution_step.num_placed_queens == n_queens:

            if not any([np.array_equiv(solution_step.board, encountered_solution) for encountered_solution in solutions]):
            
                solutions.append(solution_step.board)
                print("found a new solution!")

        # if not a valid solution then attempt placement of another piece

        else:

            # add potential solutions as appropriate

            for row, col in product(range(0, board_height), range(0, board_width)):

                is_unoccupied = not is_space_occupied(solution_step.board, row, col)
                not_threatened = is_valid_placement(solution_step.board, row, col, blocking_chars, invalid_blockers)[0]

                #print(f"{row} {col} is unoccupied: {is_unoccupied}")
                #print(f"{row} {col} not threatened: {not_threatened}")

                if (
                    is_unoccupied and not_threatened
                ):

                    # place queen

                    board_copy = solution_step.board.copy()

                    board_copy[row][col] = piece_value

                    # check whether this solution already exists in our potential solutions queue

                    #encountered_board_check = [np.array_equiv(board_copy, encountered_board) for encountered_board in encountered_boards]

                    if board_copy.__repr__() not in encountered_boards.keys():
                        
                        #print(f"piece placed at {row} {col}")
                        encountered_boards[board_copy.__repr__()] = None
                        potential_solutions.append(board_state(board_copy, solution_step.num_placed_queens + 1))

                    else:

                        # Do not add since it's already in the list of attempted solutions
                        pass
                
                else:

                    # Do not add child
                    pass

    return solutions, encountered_boards


In [56]:
trial_board = establish_board(7)
#trial_board[0][0] = 'R'
trial_board[1][1] = 'R'
trial_board[2][2] = 'R'
trial_board

array([['*', '*', '*', '*', '*', '*', '*'],
       ['*', 'R', '*', '*', '*', '*', '*'],
       ['*', '*', 'R', '*', '*', '*', '*'],
       ['*', '*', '*', '*', '*', '*', '*'],
       ['*', '*', '*', '*', '*', '*', '*'],
       ['*', '*', '*', '*', '*', '*', '*'],
       ['*', '*', '*', '*', '*', '*', '*']], dtype='<U1')

In [57]:
solutions, encountered_boards = place_queens(trial_board, trial_board.shape[0], ['R', 'Q'], ['Q'])

print(len(solutions))
for x in solutions:
    print(x)

found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a new solution!
found a ne

In [58]:
print(len(solutions))

202


In [59]:
def num_rock_enabled_solutions(
    solutions: List[np.array],
    queen_piece: str = 'Q'
) -> int:
    
    rock_enabled_solutions = {}
    
    for solution in solutions:

        # rows

        for row in range(0, solution.shape[0]):
            row_data = solution[row, :]
            if(
                np.count_nonzero(row_data == queen_piece) > 1 and
                solution.__repr__() not in rock_enabled_solutions.keys()
            ):
                rock_enabled_solutions[solution.__repr__()] = solution

        # columns

        for col in range(0, solution.shape[1]):
            column_data = solution[:, col]
            if(
                np.count_nonzero(column_data == queen_piece) > 1 and
                solution.__repr__() not in rock_enabled_solutions.keys()
            ):
                rock_enabled_solutions[solution.__repr__()] = solution

    return len(rock_enabled_solutions), list(rock_enabled_solutions.values())

In [60]:
num_rock_enabled_solutions(solutions, 'Q')

(170,
 [array([['*', 'Q', '*', '*', '*', '*', '*'],
         ['*', 'R', '*', 'Q', '*', '*', '*'],
         ['Q', '*', 'R', '*', '*', 'Q', '*'],
         ['*', '*', '*', '*', '*', '*', '*'],
         ['*', 'Q', '*', '*', '*', '*', '*'],
         ['*', '*', '*', '*', 'Q', '*', '*'],
         ['*', '*', '*', '*', '*', '*', 'Q']], dtype='<U1'),
  array([['*', '*', 'Q', '*', '*', '*', '*'],
         ['Q', 'R', '*', '*', '*', 'Q', '*'],
         ['*', '*', 'R', '*', '*', '*', '*'],
         ['*', '*', '*', '*', 'Q', '*', '*'],
         ['*', 'Q', '*', '*', '*', '*', '*'],
         ['*', '*', '*', 'Q', '*', '*', '*'],
         ['*', '*', '*', '*', '*', '*', 'Q']], dtype='<U1'),
  array([['*', '*', '*', '*', 'Q', '*', '*'],
         ['Q', 'R', 'Q', '*', '*', '*', '*'],
         ['*', '*', 'R', '*', '*', 'Q', '*'],
         ['*', '*', '*', '*', '*', '*', '*'],
         ['*', 'Q', '*', '*', '*', '*', '*'],
         ['*', '*', '*', 'Q', '*', '*', '*'],
         ['*', '*', '*', '*', '*', '*', 'Q']